### Extrair as proteínas que interagem fisicamente com o NFkB através da base de dados APIS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import hypergeom

def union(list1,list2):
    union_list = []
    for element in list1:
        if element not in union_list:
            union_list.append(element)
    for element in list2:
        if element not in union_list:
            union_list.append(element)
    return union_list

def exception(list1,list2):
    #logically substracts the second set from the first one (list1 - list2) with no repetition
    exception_list = []
    for element in list1:
        if element not in list2:
            if element not in exception_list:
                exception_list.append(element)
    return exception_list

def intersection(list1,list2):
    intersect = []
    for element in list1:
        if element in list2:
            if element not in intersect:
                intersect.append(element)
    for element in list2:
        if element in list1:
            if element not in intersect:
                intersect.append(element)
    return intersect

def unique(list1):
    unique_list = []
    for element in list1:
        if element not in unique_list:
            unique_list.append(element)
    return unique_list

## GRK: interações físicas

In [22]:
GRK5 = 'P34947'
grk_interactors = []
apid = pd.read_csv('9606_noISI_Q2.txt', sep='\t')
apid_grk_A = apid.loc[apid['UniprotID_B'] == GRK5].drop_duplicates(keep='first',subset=['UniprotID_A'])['GeneName_A'].tolist()
apid_grk_B = apid.loc[apid['UniprotID_A'] == GRK5].drop_duplicates(keep='first',subset=['UniprotID_B'])['GeneName_B'].tolist()

grk_interactors.append(apid_grk_A[0])
grk_interactors.append(apid_grk_B[0])
print(grk_interactors)


['GRK6', 'NFKBIA']


## Experimental IV: enriquecimento funcional

### Extrair as proteínas que são avlos de regulação do NFkB (i.e., cujo NFkB controla a atividade), maioritariamente através da tanscrição

In [2]:
apid = pd.read_csv('9606_noISI_Q2.txt', sep='\t')
NFKB = ['P19838','Q04206']
N1=apid.loc[apid['UniprotID_A'] == 'P19838'].drop_duplicates(keep='first',subset=['UniprotID_B'])['UniprotID_B'].tolist() #Lista de índices de proteínas que interagem com o NFKBI (p50)
R1=apid.loc[apid['UniprotID_A'] == 'Q04206'].drop_duplicates(keep='first',subset=['UniprotID_B'])['UniprotID_B'].tolist() #Lista de índices de proteínas que interagem com o NFKBI (p65)

linha_apid_1 = N1 + R1 #Lista com os indicadores uniprot de todas as proteínas que interagem com as duas subunidades do NFKB na coluna B

#Obter a lista de proteínas que interagem com o NFKB:

apid_interações_1 = apid.loc[(apid['UniprotID_B'].isin(linha_apid_1)) & (apid['UniprotID_A'].isin(NFKB))].drop_duplicates(keep='first',subset=['UniprotID_B']) #DataFrame indexado para obter apenas as proteínas que interagem com o NFkB
#_____________________________________________________________
N2=apid.loc[apid['UniprotID_B']=='P19838'].drop_duplicates(keep='first',subset=['UniprotID_A'])['UniprotID_A'].tolist()
R2=apid.loc[apid['UniprotID_B']=='Q04206'].drop_duplicates(keep='first',subset=['UniprotID_A'])['UniprotID_A'].tolist()

linha_apid_2 = N2 + R2 

apid_interações_2 = apid.loc[(apid['UniprotID_A'].isin(linha_apid_2)) & (apid['UniprotID_B'].isin(NFKB))].drop_duplicates(keep='first',subset=['UniprotID_A'])
#______________________________________________________________

#print(len(linha_apid))
linha_apid = union(linha_apid_1,linha_apid_2) #proteínas que interagem com NFkB 
apid_interações_2 = apid.loc[(apid['UniprotID_A'].isin(exception(linha_apid_2,linha_apid_1))) & (apid['UniprotID_B'].isin(NFKB))].drop_duplicates(keep='first',subset=['UniprotID_A']) #df com as proteínas que interagem com o NFKB que não aparecem em linhas_apid_1

#print(len(linha_apid_2))
#______________________________________________________________

interações_apid = pd.merge(apid_interações_1,apid_interações_2,on=['UniprotID_A','UniprotID_B'],how='outer')
selected = interações_apid[['UniprotID_A','UniprotID_B']].drop_duplicates(keep='first',subset=['UniprotID_A','UniprotID_B'])

print(f'O número de interações entre o NFKB e outras proteínas é {len(linha_apid)}')

O número de interações entre o NFKB e outras proteínas é 304


In [3]:
omni = pd.read_csv('interactions_omni.txt', sep='\t')
omni = omni.loc[omni['is_directed']==1]
target_omni_NFKBI = omni.loc[omni['source'] == NFKB[0]]['target'].tolist()
target_omni_RELA = omni.loc[omni['source'] == NFKB[1]]['target'].tolist()


#print(no_repeat)
target_omni = union(target_omni_NFKBI,target_omni_RELA)
#print(len(target_omni))

#_______________________________________________________________________________

dorothea = pd.read_csv('dorothea_AB.txt', sep='\t')

target_dorothea_NFKBI = dorothea.loc[dorothea['source'] == NFKB[0]]['target'].tolist()
target_dorothea_RELA = dorothea.loc[dorothea['source'] == NFKB[1]]['target'].tolist()

target_doro = union(target_dorothea_NFKBI,target_dorothea_RELA)
#print(len(target_doro))
alvos = union(target_omni,target_doro) # lista de proteínas que são reguladas pelo NFkB (maioritariamente através da transcrição)


### Extrair a lista de proteínas que regulam o NFkB

In [4]:
source_omni_NFKBI = omni.loc[omni['target'] == NFKB[0]]['source'].tolist()
source_omni_RELA = omni.loc[omni['target'] == NFKB[1]]['source'].tolist()

source_omni = union(source_omni_NFKBI,source_omni_RELA)
#________________________________________________________________

source_dorothea_NFKBI = dorothea.loc[dorothea['target'] == NFKB[0]]['source'].tolist()
source_dorothea_RELA = dorothea.loc[dorothea['target'] == NFKB[1]]['source'].tolist()

source_dorothea = union(source_dorothea_NFKBI,source_dorothea_RELA)

reguladores = union(source_omni,source_dorothea)

# Intersecção da lista de reguladores e alvos

alvos_e_reg = intersection(alvos,reguladores)           # Possíveis indicadores de retroação negativa
alvos_e_interactores = intersection(linha_apid,alvos)   # Possíveis indicadores de retroação negativa (muitos interatores também são reguladores)
interactores_e_reg = intersection(linha_apid,reguladores)
#print(len(alvos_e_reg),len(reg_e_interactores),len(interactores_e_alvos))
a_i_r = intersection(alvos_e_interactores,alvos_e_reg)  #proteínas que são, simultaneamente, alvos, reguladores e interatores do NFkB. Explorar!
print(a_i_r)


['Q04206', 'Q13547', 'P41279', 'P19838', 'P25963', 'Q15653', 'P14921', 'O14920', 'Q15025', 'Q9Y6K9', 'O15111', 'P01100', 'P08047', 'P42224', 'P04150', 'P04637', 'P20749', 'P68400', 'Q96EB6', 'Q92793', 'Q9Y2K7', 'P38398']


### Análise das interações obtidas a partir de um teste hipergeométrico

Identificar se essas interações são "normais" para o conjunto de dados obtidos

In [5]:
todos_apid = union(apid['UniprotID_A'].tolist(),apid['UniprotID_B'].tolist())

In [6]:
#print(len(todos_apid))
todos_omni = union(omni['source'].tolist(),omni['target'].tolist())
todos_dorothea = union(dorothea['source'].tolist(),dorothea['target'].tolist())
todos = union(union(todos_dorothea,todos_omni),todos_apid) # o universo do teste que será usado: todas as proteínas de todas as bases de dados consultadas
#print(len(todos_omni),len(todos_dorothea),len(todos))

In [7]:
t = len(todos)
a = len(alvos)
r = len(reguladores)
i = len(linha_apid)
a_i=len(alvos_e_interactores)
a_r=len(alvos_e_reg)
r_i=len(interactores_e_reg)

print(a,a_i,i,t)

247 42 304 19147


### Calcular a probabilidade das intersecções obtidas ocorrerem para o universo t obtido
A classe hypergeo cria um objeto cujas 'instance variables' são os parâmetros da distribuição hypergeométrica definidos pelo universo, total de 'bolas brancas' e de 'bolas retiradas'. O cálculo do p_value envolve encontrar a probabilidade para que haja tantas intersecções quantas as observadas (i.e., das n bolas retiradas, x serão brancas, sendo x o número de intersecções e n o número do conjunto de proteínas com maior elemento entre ambos a serem comparados) ou mais, i.e.:
- p_value = P(X>=x) 

In [8]:
from decimal import Decimal

def factorial(n):
    result = 1
    for i in range(1, int(n)+1):
        result *= i
    return result

def combination(x,y):
    numerator = Decimal(factorial(x))
    denominator = Decimal(factorial(y)*factorial(x-y))
    comb = numerator/denominator
    #comb = factorial(x)/(factorial(y)*factorial(x-y))
    return comb

# N bolas, b brancas, p = n-b pretas, r retiradas, x brancas das retiradas (evento favorável), r-x pretas das retiradas
class hypergeo():
    def __init__(self,total,white,drawn):
        self.total = total
        self.white = white
        self.drawn = drawn

    def mass_function(self,x,plot=True):
        variable = np.arange(0,x+1,1)
        probabilities = []
        for number in variable:
            probability = combination(self.white,number)*combination(self.total-self.white,self.drawn-number)/combination(self.total,self.drawn)
            probabilities.append(probability)
        if plot==False:
            return probabilities
        else:
            f, ax = plt.subplots(1,1,figsize=(7,5))
            return ax.bar(variable,probabilities)

    def p_val(self,x,lower_tail=False):
        #Calculates the upper tail probability for a given set of values (P(X>=x))
        if lower_tail == False:
            variable = np.arange(x,self.white+1,1)
            probabilities = (combination(self.white,number)*combination(self.total-self.white,self.drawn-number)/combination(self.total,self.drawn) for number in variable)
            return sum(probabilities)
        else:
        #Calculates the lower tail probability for a given set of values (P(X<x)). DOES NOT include the x value.
            variable = np.arange(0,x,1)
            probabilities = (combination(self.white,number)*combination(self.total-self.white,self.drawn-number)/combination(self.total,self.drawn) for number in variable)
            return sum(probabilities)

#p = hypergeo(20,7,12).p_val(4+1,lower_tail=True)

p_a_i = hypergeo(t,a,i).p_val(a_i)
p_a_r = hypergeo(t,a,r).p_val(a_r)
p_r_i = hypergeo(t,r,i).p_val(r_i)

print(f'The p_value for p_a_i is {p_a_i}, for p_a_r is {p_a_r} and for {p_r_i}')

The p_value for p_a_i is 4.952881556495031423205119583E-31, for p_a_r is 1.703591408486587656001120691E-34 and for 1.668784781679130923648604418E-63


In [9]:
print(f'The p_value for p_a_i is {p_a_i}, for p_a_r is {p_a_r} and for p_r_i is {p_r_i}')

The p_value for p_a_i is 4.952881556495031423205119583E-31, for p_a_r is 1.703591408486587656001120691E-34 and for p_r_i is 1.668784781679130923648604418E-63
